In [7]:
import pandas as pd
import numpy as np
import random

### hotel_after 전처리 코드

In [6]:
# CSV 파일 읽기
room_df = pd.read_csv('../data/room.csv', encoding='utf-8-sig')
hotel_after_df = pd.read_csv('../data/hotel_after.csv', encoding='utf-8-sig')

# offseasonminnfee의 가장 작은 값을 찾기 위해 contentid로 그룹화
min_fees = room_df.groupby('contentid')['offseasonminnfee'].min().reset_index()

# min_fees 데이터프레임의 컬럼 이름 변경
min_fees.rename(columns={'offseasonminnfee': 'minfee'}, inplace=True)

# hotel_after_df와 min_fees 데이터프레임을 contentid 기준으로 병합
merged_df = hotel_after_df.merge(min_fees, on='contentid', how='left')

# minfee 값을 업데이트
hotel_after_df['minfee'] = merged_df['minfee']

# 업데이트된 hotel_after_df를 새로운 CSV 파일로 저장
hotel_after_df.to_csv('../data/hotel_after_updated.csv', index=False, encoding='utf-8-sig')

print("최소 요금이 업데이트된 hotel_after_updated.csv 파일이 생성되었습니다.")


최소 요금이 업데이트된 hotel_after_updated.csv 파일이 생성되었습니다.


### room 전처리 코드

In [25]:
import pandas as pd
import numpy as np
import random

# CSV 파일 읽기
df = pd.read_csv('../data/room_detail_raw.csv')

# 열 이름 변경
df.rename(columns={
    'availablecount': 'available_count',
    'roombasecount': 'base_person',
    'roommaxcount': 'max_person',
    'roomoffseasonminfee1': 'offseasonminfee',
    'roomoffseasonminfee2': 'offseasonmaxfee',
    'roompeakseasonminfee1': 'peakseasonminfee',
    'roompeakseasonminfee2': 'peakseasonmaxfee',
    'roomimg1alt': 'roomimg1_alt',
    'roomimg2alt': 'roomimg2_alt',
    'roomimg3alt': 'roomimg3_alt'
}, inplace=True)

# 중복 행 제거
df = df.drop_duplicates(subset=['contentid', 'roomcode'])

# roomcount 값 처리
df['roomcount'] = np.where(df['roomcount'] == 0, np.random.randint(0, 20, size=len(df)), df['roomcount'])


# 이미 존재하는 roomcode와 contentid 목록 생성
existing_roomcodes = set(df['roomcode'].dropna().unique())
existing_contentids = set(df['contentid'].dropna().unique())

# roomcode가 비어있는 경우 처리
for index, row in df.iterrows():
    if pd.isnull(row['roomcode']) or row['roomcode'] == '':
        while True:
            # 5~6자리 숫자 랜덤 roomcode 생성
            length = random.randint(5, 6)
            new_roomcode = ''.join(random.choices('0123456789', k=length))
            
            # 새 roomcode가 기존 roomcode와 contentid에 없는지 확인
            if new_roomcode not in existing_roomcodes and new_roomcode not in existing_contentids:
                df.at[index, 'roomcode'] = new_roomcode  # 새로운 roomcode를 데이터프레임에 추가
                existing_roomcodes.add(new_roomcode)  # 새 roomcode를 기존 목록에 추가
                break  # 중복되지 않으면 루프 종료

df['roomcode'] = df['roomcode'].astype(int)

# available_count 계산
df['available_count'] = np.where(df['roomcount'] > 0, df['roomcount'] - np.random.randint(0, df['roomcount'] + 1, size=len(df)), 0)

# 기본 인원 수 및 최대 인원 수 처리
df['base_person'] = np.where(df['base_person'] == 0, np.random.randint(0, 25, size=len(df)), df['base_person'])
df['max_person'] = df['base_person'] + np.random.randint(0, 6, size=len(df))

# 요금 처리
df['offseasonminfee'] = np.where(df['offseasonminfee'] == 0, np.random.randint(200, 1000, size=len(df)) * 100, df['offseasonminfee'])
df['offseasonmaxfee'] = np.where((df['offseasonmaxfee'].isnull()) | (df['offseasonmaxfee'] == 0) | (df['offseasonmaxfee'] <= df['offseasonminfee']), df['offseasonminfee'] + np.random.randint(5, 50, size=len(df)) * 1000, df['offseasonmaxfee'])
df['peakseasonminfee'] = np.where((df['peakseasonminfee'].isnull()) | (df['peakseasonminfee'] == 0) | (df['peakseasonminfee'] <= df['offseasonminfee']), df['offseasonminfee'] + np.random.randint(5, 50, size=len(df)) * 1000, df['peakseasonminfee'])
df['peakseasonmaxfee'] = np.where((df['peakseasonmaxfee'].isnull()) | (df['peakseasonmaxfee'] == 0) | (df['peakseasonmaxfee'] <= df['peakseasonminfee']), df['peakseasonminfee'] + np.random.randint(5, 50, size=len(df)) * 1000, df['peakseasonmaxfee'])

# 불필요한 열 삭제
drop_columns = ['contenttypeid', 'roomsize1', 'roomintro', 'cpyrhtDivCd1', 'cpyrhtDivCd2', 'cpyrhtDivCd3', 'roomimg4', 'roomimg4alt', 'cpyrhtDivCd4', 'roomimg5', 'roomimg5alt', 'cpyrhtDivCd5', 'roomsize2']
df = df.drop(columns=drop_columns)

# 결측값 처리
df.fillna('', inplace=True)

# 열 순서 변경
new_order = [
    'contentid', 'roomcode', 'roomtitle', 'roomcount', 'available_count',
    'base_person', 'max_person', 'offseasonminfee', 'offseasonmaxfee',
    'peakseasonminfee', 'peakseasonmaxfee', 'roombathfacility', 'roombath',
    'roomhometheater', 'roomaircondition', 'roomtv', 'roompc', 'roomcable',
    'roominternet', 'roomrefrigerator', 'roomtoiletries', 'roomsofa',
    'roomcook', 'roomtable', 'roomhairdryer', 'roomimg1', 'roomimg1_alt',
    'roomimg2', 'roomimg2_alt', 'roomimg3', 'roomimg3_alt'
]

# 데이터프레임의 열 순서 변경
df = df[new_order]

# 최종 데이터프레임을 CSV로 저장
df.to_csv('../data/room.csv', index=False, encoding='utf-8-sig')


C:\Users\cho03\AppData\Local\Temp\ipykernel_4436\3286293353.py:43: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '39690' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.at[index, 'roomcode'] = new_roomcode  # 새로운 roomcode를 데이터프레임에 추가
